## Checking Constrained Dynamics

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import symd

In [ ]:
symd.prepare_input(15, 2, 8, "wp-15")

In [ ]:
with open("wp-15.json") as f:
    group = json.load(f)
    projector = group["projector"]

with open("wp-15-01.json") as f:
    group = json.load(f)
e = np.array(group["members"][0]).reshape(3, 3)
# e = np.array([0, 0, 0, 0., 0, 0, 0, 0, 1]).reshape(3,3)
basis = np.array(projector).reshape(4, 4) @ np.array([50, 0, 0, 50])
basis = basis.reshape(2, 2)
# basis = np.eye(2)
ib = np.linalg.inv(basis)
print(e)
print(ib)
print(basis)
print(ib @ basis)

In [ ]:
def project(projector, basis):
    return np.array(projector).reshape(4, 4) @ basis.flatten()


basis = np.array([50, 0, 0, 50])
project(projector, basis), project(projector, project(projector, basis))

In [ ]:
def scale(x, ib=ib):
    s = ib @ x
    return np.fmod(s, 1.0)


def unscale(x, basis=basis):
    return basis @ x


x = np.array([0.5, 0.5])
print(x)
print(unscale(x))
print(scale(unscale(x)))

In [ ]:
def pa(x, e=e):
    return (e @ (np.concatenate((x, [1])).T))[:2]


pa(x)

In [ ]:
# compute A matrix
N = 2
A = np.zeros((N, N))
ibt = ib
grad = e[:-1, :-1] @ ibt - ibt
print("grad", grad)
print(A)
A = grad @ grad
print(A)
Ainv = np.linalg.pinv(A)
print(Ainv)
print(Ainv @ A)
Ainv = Ainv @ grad
print(Ainv)

In [ ]:
def step(x, v, f, dt, gamma=0.05, T=0.2):
    # b
    # v += f(x) * dt / 2
    # a
    x += v * dt / 2
    # constraint 1
    s = scale(x)
    delta = pa(s) - s
    print("delta:", delta, "s", s, "x", x)
    l = Ainv @ delta.T
    print("lambda", l)
    v -= l / dt * 2
    x -= l
    s = scale(x)
    delta = pa(s) - s
    print("after delta:", delta, "s", s, "x", x)
    # o
    c1 = np.exp(-gamma * dt)
    c2 = np.sqrt(1 - c1**2)
    v *= c1
    v += c2 * np.random.normal() * np.sqrt(T)
    # a
    x += dt * v / 2
    # constraint 2
    s = scale(x)
    delta = pa(s) - s
    print("delta:", delta, "s", s, "x", x)
    l = Ainv @ delta.T
    print("A lambda", A @ l)
    v -= l / dt * 2
    x -= l
    s = scale(x)
    delta = pa(s) - s
    print("after delta:", delta, "s", s, "x", x)
    # b
    v += f(x) * dt / 2
    return x, v


x = np.array([0.376419008701368, 0.008526634164955138], dtype=float)
x = unscale(x)
f = lambda x: -2 * np.sqrt(np.sum(x**2)) * x
v = np.array([0.5, 1], dtype=float)
N = 5
traj = np.empty((N, 2), dtype=float)
vtraj = np.empty((N, 2), dtype=float)
for i in range(N):
    traj[i, :] = x
    vtraj[i, :] = v
    x, v = step(x, v, f, 0.05)
    print(x)

In [ ]:
plt.plot(traj[:, 0], traj[:, 1], ".-")
# for i in range(N):
#     xi = traj[i]
#     vi = vtraj[i]
#     vi /= np.linalg.norm(vi) / 0.2
#     plt.plot([xi[0], xi[0] + vi[0]], [xi[1], xi[1] + vi[1]], '-', color='C1')
plt.gca().axis("equal")

In [ ]:
traj

## Checking Symmetry of Proposal Distribution

In [ ]:
sign = lambda x: bool(x > 0) - bool(x < 0)


def levi_civta(index):
    p = 1
    d = len(index)
    for i in range(d):
        for j in range(i + 1, d):
            p *= sign(index[j] - index[i])
    return p


def rvolume(b, v, i, index):
    d = len(index)
    if i == d:
        return levi_civta(index) * v
    vi = 0
    for j in range(d):
        index[i] = j
        vi += rvolume(b, b[j][i] * v, i + 1, index)
    return vi


def volume(b):
    index = [0] * len(b)
    return rvolume(b, 1, 0, index)

In [ ]:
def proj(ub):
    fub = ub.flatten()
    fb = np.array(projector).reshape(4, 4) @ fub
    return fb.reshape(2, 2)


vi = []
vj = []
for i in range(10000):
    ub = np.array([[1, 0], [0, 1]]) + np.random.normal(size=(2, 2)) * 0.1
    vi.append(volume(proj(ub)))
    ub *= (1 + 0.01 - np.random.uniform(size=4) * 0.02).reshape(2, 2)
    vj.append(volume(proj(ub)))
vi, vj = np.array(vi), np.array(vj)

In [ ]:
plt.hist(vj / vi, bins=np.linspace(0.8, 1.2, 25), alpha=0.7)
plt.hist(vi / vj, bins=np.linspace(0.8, 1.2, 25), alpha=0.7)
plt.show()

In [ ]:
np.mean(vj / vi), np.mean(vi / vj)

In [ ]:
plt.plot(vj, vi, ".")
plt.plot(vi, vj, ".")

## Reflection

In [ ]:
def triangle_wave(x):
    # return 2 * np.abs(np.abs(np.fmod(x + 0.5, 1.)) - 1/2)
    return 2 * np.abs(x / 2 - np.floor(x / 2 + 1 / 2))
    # return np.fmod(x, 1.0)


def square_wave(x):
    return 2 * (2 * np.floor(x / 2) - np.floor(x)) + 1


x = np.linspace(-0.5, 1.5, 10000)
plt.plot(x, triangle_wave(x))
plt.plot(x, square_wave(x))
# plt.ylim(0,1)

In [ ]:
triangle_wave(0.4)